### Common Crawl statistics

##### Refrences
* List of index files: https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-09/cc-index.paths.gz
* Index: https://commoncrawl.s3.amazonaws.com/cc-index/collections/CC-MAIN-2019-09/indexes/cdx-00100.gz (0.8Gb)
* WARC file: https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-09/segments/1550247482347.44/warc/CC-MAIN-20190217172628-20190217194628-00620.warc.gz (1.0Gb)
* [The WARC Format 1.1](http://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/)

In [1]:
import gzip

In [3]:
gz = gzip.open('/Users/aromanov/Downloads/CC-MAIN-20190217172628-20190217194628-00620.warc.gz', 'rt')

In [ ]:
l = gz.readline()